In [2]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as panda

standingsUrl = "https://fbref.com/en/comps/25/2022/2022-J1-League-Stats"
data = requests.get(standingsUrl)
parsedData = BeautifulSoup(data.text)
tableData = parsedData.select("table.stats_table")[0]
teamLinks = tableData.find_all("a")
links = [link.get("href") for link in teamLinks]
linksFiltered = [link for link in links if "/squads/" in link]
teamUrls = [f"https://fbref.com/{link}" for link in linksFiltered]
allTeamMatches = []

for url in teamUrls:
    teamName = url.split("/")[-1].replace("-Stats", "")

    teamData = requests.get(url)
    allMatches = panda.read_html(teamData.text, match="Scores & Fixtures")[0]
    allMatches["team"] = url.split("/")[-1].replace("-Stats", "")
    playedMatches = allMatches[(allMatches.Result.notnull())]

    shootingStats = BeautifulSoup(teamData.text)
    shootingLinks = shootingStats.find_all("a")
    shootingLinks = [link.get("href") for link in shootingLinks]
    shootingLinks = [link for link in shootingLinks if link and "all_comps/shooting/" in link]
    shootingData = requests.get(f"https://fbref.com{shootingLinks[0]}")
    shooting = panda.read_html(shootingData.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    
    try:
        teamData = playedMatches.merge(shooting[["Date", "Sh", "SoT", "PK", "PKatt"]], on="Date")
    except ValueError:
        continue


    allTeamMatches.append(teamData)
    time.sleep(1)
        
    

In [5]:
matchDataframe = panda.concat(allTeamMatches)
matchDataframe.columns = [c.lower() for c in matchDataframe.columns]
matchDataframe = matchDataframe.sort_values(by="date", ascending=True)
matchDataframe.to_csv("matches2022-23.csv")

In [6]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as panda

standingsUrl = "https://fbref.com/en/comps/25/J1-League-Stats"
data = requests.get(standingsUrl)
parsedData = BeautifulSoup(data.text)
tableData = parsedData.select("table.stats_table")[0]
teamLinks = tableData.find_all("a")
links = [link.get("href") for link in teamLinks]
linksFiltered = [link for link in links if "/squads/" in link]
teamUrls = [f"https://fbref.com/{link}" for link in linksFiltered]
allTeamMatches = []

for url in teamUrls:
    teamName = url.split("/")[-1].replace("-Stats", "")

    teamData = requests.get(url)
    allMatches = panda.read_html(teamData.text, match="Scores & Fixtures")[0]
    allMatches["team"] = url.split("/")[-1].replace("-Stats", "")
    playedMatches = allMatches[(allMatches.Result.notnull())]

    shootingStats = BeautifulSoup(teamData.text)
    shootingLinks = shootingStats.find_all("a")
    shootingLinks = [link.get("href") for link in shootingLinks]
    shootingLinks = [link for link in shootingLinks if link and "all_comps/shooting/" in link]
    shootingData = requests.get(f"https://fbref.com{shootingLinks[0]}")
    shooting = panda.read_html(shootingData.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    
    try:
        teamData = playedMatches.merge(shooting[["Date", "Sh", "SoT", "PK", "PKatt"]], on="Date")
    except ValueError:
        continue


    allTeamMatches.append(teamData)
    time.sleep(1)
        
    

In [7]:
matchDataframe = panda.concat(allTeamMatches)
matchDataframe.columns = [c.lower() for c in matchDataframe.columns]
matchDataframe = matchDataframe.sort_values(by="date", ascending=True)
matchDataframe.to_csv("matches2023-24.csv")

In [8]:
data2022 = panda.read_csv("matches2022-23.csv")

In [9]:
data2023 = panda.read_csv("matches2023-24.csv")

In [10]:
totalData = panda.concat([data2022, data2023])

In [11]:
totalData.to_csv("totalMatches.csv")